# Qiskit Assignment 3
## Multiple Qubits, Entanglement, and Teleportation

### Learning Objectives
1. Construct circuits with multiple qubits
2. Construct circuits with entangled states
3. Implement teleportation and understand the role of an eavesdropper

In [ ]:
# Import Qiskit and other needed packages
from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit_textbook.tools import array_to_latex
import random
import numpy as np
import pprint

#### Task 1 (1a, 1b) - Multiple Qubit Circuits

#### Task 1a
Run the following cell. Return a circuit from `multi_qubit_a` that produces the following target state on measurement. It's okay if your resulting state is equivalent up to a global phase. Your qubits must start in state $|00\rangle$ i.e. no initializations are allowed.

In [ ]:
array_to_latex([-1j/2, -1/2, 1/2, -1j/2], pretext="\\text{Target Statevector} = ")

In [ ]:
def multi_qubit_a():
    qc_a = QuantumCircuit(2,2)
    # BEGIN SOLUTION
    # hint: recognize the target state is tensor factorable so this is not an entangled state.
    # So we can find what states the qubits need to be in individually.
    # If we prepare |0> on qubit zero and |1> on qubit one,
    # can use outer product from lecture 01 to obtain the matrix necessary to make this transformation
    # they've practiced using the U gate in q1 assignment so can solve for the corresponding U gate parameters.
    #
    # primary solution - U gate
    phi = 3*np.pi/2
    theta = np.pi/2
    lamda = np.pi/2
    qc_a.u(phi,theta,lamda,0)
    qc_a.u(phi,theta,lamda,1)
    qc_a.x(1)
    # alternate solution - P, H, and X gates
#     qc_a.h(0)
#     qc_a.p(3*np.pi/2,0)
#     qc_a.h(1)
#     qc_a.p(3*np.pi/2,1)
#     qc_a.x(1)
    qc_a.measure(0,0)
    qc_a.measure(1,1)
    # END SOLUTION
    qc_a.reverse_bits() # remember to account for Qiskit's reversed qubit ordering!
    return qc_a


In [ ]:
multi_qubit_a().draw(output='mpl')

In [ ]:
backend = BasicAer.get_backend("statevector_simulator")
qc = multi_qubit_a()
qc.remove_final_measurements()
ket = Statevector.from_instruction(qc)
array_to_latex(ket, pretext="\\text{Circuit Statevector} = ")

In [ ]:
def testNoInitializations_1a():
    ops = multi_qubit_a().count_ops()
    try:
        find_z = ops['initialize']
    except KeyError:
        return True
    else:
        return False
    
testNoInitializations_1a()

In [ ]:
def testMeasurements_1a():
    ops = multi_qubit_a().count_ops()
    return ops['measure'] == 2

testMeasurements_1a()

In [ ]:
def testAmplitudes_1a():
    return Statevector([-1j/2, -1/2, 1/2, -1j/2]).equiv(ket)

testAmplitudes_1a()

#### Task 2 (2A, 2B) - What's in the box?

Build a circuit that figures out what rotation is done by an oracle. 

**README**: Your solutions to 2A and 2B may **NOT** use conditional statements to dynamically pick gates according to the input or otherwise attempt to influence the oracle. In other words, the only gate that may vary between runs is the oracle, and do **NOT** use the parameter `r` in any code you write. Solutions not adhering to these guidelines will be ***severely penalized***.

#### Task 2A
The oracle promises to be **I** or **Z**. Return a circuit from `whats_in_box_a` such that 
- measurement yields either $|0\rangle$ or $|1\rangle$
- the probability of measuring each state is correlated perfectly with the gates returned by the oracle
- example: if you return $|1\rangle$ for an oracle of Z, the probability of seeing $|1\rangle$ given I should be 0.

In [ ]:
def oracle_a(qc, r=None):
    if r is None:
        r = random.uniform(0, 1)
    qc.i(0) if r > 0.5 else qc.z(0)
    return qc

def whats_in_box_a(r=None):
    # BEGIN SOLUTION
    qc = QuantumCircuit(1,1)
    qc.h(0)
    # END SOLUTION
    qc.barrier()
    qc = oracle_a(qc,r)
    qc.barrier()
    # BEGIN SOLUTION
    qc.h(0)
    qc.measure(0,0)
    # END SOLUTION
    return qc

qc = whats_in_box_a()
qc.draw(output='mpl')

In [ ]:
qc.remove_final_measurements()
ket = Statevector.from_instruction(qc)
array_to_latex(ket, pretext="\\text{2A last run} = ")

#### Task 2b 
The oracle promises to be **I**, **X**, **Y**, or **Z**. Return a circuit from `whats_in_box_b` such that
- measurement yields either $|00\rangle$, $|01\rangle$, $|10\rangle$, or $|11\rangle$
- the probability of seeing each state is correlated perfectly with specific gates returned by the oracle
- example: if you return $|01\rangle$ for an oracle of Z, the probability of seeing $|01\rangle$ given any other gate should be 0.

In [ ]:
def oracle_b(qc, r=None):
    if r is None:
        r = random.uniform(0, 1)
    if r < 0.25:
        qc.i(0)
        qc.i(1)
    elif r < 0.5:
        qc.x(0)
        qc.x(1)
    elif r < 0.75:
        qc.y(0)
        qc.y(1)
    else:
        qc.z(0)
        qc.z(1)
    return qc

def whats_in_box_b(r=None):
    # BEGIN SOLUTION
    qc = QuantumCircuit(2,2)
    qc.h(0)
    # END SOLUTION
    qc.barrier()
    qc = oracle_b(qc,r)
    qc.barrier()
    # BEGIN SOLUTION
    qc.h(0)
    qc.measure(0,0)
    qc.measure(1,1)
    # END SOLUTION
    qc.reverse_bits()
    return qc

qc = whats_in_box_b()
qc.draw(output='mpl')

In [ ]:
qc.remove_final_measurements()
ket = Statevector.from_instruction(qc)
array_to_latex(ket, pretext="\\text{2A last run} = ")

In [ ]:
def testValidateRulesA():
    qc_z = whats_in_box_a(0)
    qc_i = whats_in_box_a(1)
    ops_z = qc_z.count_ops()
    ops_i = qc_i.count_ops()
    if abs(len(ops_z) - len(ops_i)) > 1:
        return False
    for (k, v) in ops_z.items():
        if k in ops_i:
            if v != ops_i[k]:
                if abs(ops_i[k] - v) > 0 and (k != 'z' and k != 'i'):
                    return False
    try:
        nz = ops_i['z']
    except KeyError:
        nz = 0
    try:
        ni = ops_z['id']
    except KeyError:
        ni = 0
    return True if ops_i['id'] - ni == 1 and ops_z['z'] - nz == 1 else False
    
testValidateRulesA()

In [ ]:
def testZOracleA():
    sim = BasicAer.get_backend("qasm_simulator")
    shots = 100
    qc = whats_in_box_a(0)
    job = execute(qc, sim, shots=shots)
    counts = job.result().get_counts()
    res = counts['0'] if '0' in counts else counts['1']
    return len(counts) == 1 and res == shots and testValidateRulesA()

testZOracleA()

In [ ]:
def testIOracleA():
    sim = BasicAer.get_backend("qasm_simulator")
    shots = 100
    qc = whats_in_box_a(1)
    job = execute(qc, sim, shots=shots)
    counts = job.result().get_counts()
    res = counts['0'] if '0' in counts else counts['1']
    return len(counts) == 1 and res == shots and testValidateRulesA()

testIOracleA()

In [ ]:
def testMixedOracleA():
    trials = 5
    res = [testZOracleA() and testIOracleA() for i in range(trials)]
    return all(res)

testIOracleA()

## Conclusion
Any finals thoughts/summary go here...

### Extension Ideas
1. Solve the twin prime conjecture
2. Count down from infinity
3. etc...